In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from matplotlib.patches import Circle, Wedge
from matplotlib.axes import Axes
from matplotlib.lines import Line2D
from shapely.geometry import Point
from shapely.ops import unary_union
from matplotlib.animation import FuncAnimation
from ipywidgets import interact_manual, IntSlider, FloatSlider

# -----------------------------
# Algorithm parameters
# -----------------------------
ALG_PARAMS = {
    "n_points": 50,
    "canvas_width": 150,
    "canvas_height": 100,
    "scan_radius": 10,
    "min_points": 3,
}

# -----------------------------
# Visual style parameters
# -----------------------------
POINT_STYLE = {
    "size": 30,
    "core_color": "darkred",
    "reachable_color": "lightcoral",
    "unclassified_color": "gray",
    "noise_color": "black",
}

CIRCLE_STYLE = {"border_color": "black", "border_width": 1.0}
LINK_STYLE = {"color": "green", "width": 1.0, "linestyle": "--"}
REGION_STYLE = {"color": "black", "width": 2}

# -----------------------------
# Core computation functions
# -----------------------------
def find_core_points(points, scan_radius, min_points):
    core_indices = []
    points_in_circle = []
    for i, p in enumerate(points):
        neighbors = np.where(np.linalg.norm(points - p, axis=1) <= scan_radius)[0]
        if len(neighbors) >= min_points:
            core_indices.append(i)
            points_in_circle.append(set(neighbors))
        else:
            points_in_circle.append(set([i]))
    return points[core_indices], core_indices, points_in_circle

def build_core_graph(core_indices, points_in_circle):
    G = nx.Graph()
    for idx in range(len(core_indices)):
        G.add_node(idx)
    for i, ci in enumerate(core_indices):
        for j, cj in enumerate(core_indices):
            if i >= j:
                continue
            if len(points_in_circle[ci] & points_in_circle[cj]) > 0:
                G.add_edge(i, j)
    return G

def find_reachable_and_noise(points, core_indices, G, core_points, scan_radius):
    reachable = set()
    for component in nx.connected_components(G):
        circles = [
            Point(core_points[i]).buffer(scan_radius, resolution=64) for i in component
        ]
        union_shape = unary_union(circles)
        for i, p in enumerate(points):
            if i not in core_indices and union_shape.contains(Point(p)):
                reachable.add(i)
    noise = set(range(len(points))) - set(core_indices) - reachable
    return reachable, noise

# -----------------------------
# Plotting helpers
# -----------------------------
def base_axis(ax, canvas_width, canvas_height):
    ax.set_xlim(0, canvas_width)
    ax.set_ylim(0, canvas_height)
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])

def plot_all_points(ax, points, core_indices=None, reachable=None, noise=None):
    ax.scatter(
        points[:, 0],
        points[:, 1],
        s=POINT_STYLE["size"],
        color=POINT_STYLE["unclassified_color"],
        zorder=1,
    )
    if core_indices:
        ax.scatter(
            points[core_indices, 0],
            points[core_indices, 1],
            s=POINT_STYLE["size"],
            color=POINT_STYLE["core_color"],
            zorder=4,
        )
    if reachable:
        ax.scatter(
            points[list(reachable), 0],
            points[list(reachable), 1],
            s=POINT_STYLE["size"],
            color=POINT_STYLE["reachable_color"],
            zorder=4,
        )
    if noise:
        ax.scatter(
            points[list(noise), 0],
            points[list(noise), 1],
            s=POINT_STYLE["size"],
            color=POINT_STYLE["noise_color"],
            marker="x",
            zorder=5,
        )

def plot_core_circles(ax, core_points, scan_radius):
    for p in core_points:
        ax.add_patch(
            Circle(
                p,
                scan_radius,
                fill=False,
                edgecolor=CIRCLE_STYLE["border_color"],
                linewidth=CIRCLE_STYLE["border_width"],
                zorder=3,
            )
        )

def plot_core_links(ax, G, core_points):
    for i, j in G.edges():
        p1, p2 = core_points[i], core_points[j]
        ax.plot(
            [p1[0], p2[0]],
            [p1[1], p2[1]],
            color=LINK_STYLE["color"],
            linewidth=LINK_STYLE["width"],
            linestyle=LINK_STYLE["linestyle"],
            zorder=2,
        )

def plot_connected_regions(ax, G, core_points, scan_radius):
    for component in nx.connected_components(G):
        circles = [
            Point(core_points[i]).buffer(scan_radius, resolution=64) for i in component
        ]
        union_shape = unary_union(circles)
        polygons = (
            [union_shape]
            if union_shape.geom_type == "Polygon"
            else list(union_shape.geoms)
        )
        for poly in polygons:
            x, y = poly.exterior.xy
            ax.plot(
                x,
                y,
                color=REGION_STYLE["color"],
                linewidth=REGION_STYLE["width"],
                zorder=6,
            )

def add_dynamic_legend(ax, show_core_indices:bool=False, show_reachable:bool=False, show_noise:bool=False,
                       show_circles:bool=False, show_links:bool=False, show_regions:bool=False):
    handles = []

    handles.append(Line2D([0], [0], marker="o", color="w",
                          label="Unclassified",
                          markerfacecolor=POINT_STYLE["unclassified_color"],
                          markersize=8))
    if show_core_indices:
        handles.append(Line2D([0], [0], marker="o", color="w",
                              label="Core",
                              markerfacecolor=POINT_STYLE["core_color"],
                              markersize=8))
    if show_reachable:
        handles.append(Line2D([0], [0], marker="o", color="w",
                              label="Reachable",
                              markerfacecolor=POINT_STYLE["reachable_color"],
                              markersize=8))
    if show_noise:
        handles.append(Line2D([0], [0], marker="x",
                              color=POINT_STYLE["noise_color"],
                              label="Noise"))
    if show_circles:
        handles.append(Line2D([0], [0],
                              color=CIRCLE_STYLE["border_color"],
                              lw=CIRCLE_STYLE["border_width"],
                              label="Core Circle"))
    if show_links:
        handles.append(Line2D([0], [0],
                              color=LINK_STYLE["color"],
                              lw=LINK_STYLE["width"],
                              linestyle=LINK_STYLE["linestyle"],
                              label="Core Link"))
    if show_regions:
        handles.append(Line2D([0], [0],
                              color=REGION_STYLE["color"],
                              lw=REGION_STYLE["width"],
                              label="Connected Region"))

    if len(handles) > 0:
        ax.legend(handles=handles, loc="upper right")

# -----------------------------
# Radar scan animation
# -----------------------------
animations = []  # global list to keep animation objects alive


def radar_animation(ax:plt.Axes,points, min_points,
                    scan_radius, sweep_angle=60, sweep_speed=5, n_fade=10, ):
    """
    Animate radar scan for each point as center.
    Core points inside the scanning wedge are highlighted.

    Parameters:
    - points: numpy array (N,2), coordinates of all points
    - min_points: minimum neighbors to qualify as core point
    - scan_radius: scanning radius
    - sweep_angle: angular width of wedge (degrees)
    - sweep_speed: rotation speed per frame (degrees)
    - n_fade: number of gradient wedges for fading effect

    Returns:
    - ani: FuncAnimation object
    """
    fig = ax.figure

    # Initialize scatter points
    colors = [POINT_STYLE['unclassified_color']] * len(points)
    scat = ax.scatter(points[:,0], points[:,1], s=POINT_STYLE['size'], color=colors, zorder=4)

    # Radar boundary circle
    radar_circle = Circle((0,0), scan_radius, fill='lightgreen', color='lightgreen', lw=2)
    ax.add_patch(radar_circle)

    # Gradient wedges
    wedges = [Wedge((0,0), scan_radius, 0, sweep_angle*(i+1)/n_fade,
                    facecolor='green', alpha=(i+1)/n_fade*0.4) for i in range(n_fade)]
    
    for w in wedges:
        ax.add_patch(w)

    # Store confirmed core points and their circles
    confirmed_core_indices = set()
    confirmed_reachable_indices = set()
    core_circles = {}  # Dictionary to store core point circles

    n_points = len(points)
    frames_per_point = int(360 / sweep_speed)
    total_frames = n_points * frames_per_point

    def update(frame):
        center_idx = frame // frames_per_point
        frame_in_circle = frame % frames_per_point
        theta_start = frame_in_circle * sweep_speed

        center = points[center_idx]
        radar_circle.center = center

        # Update wedges
        for i, w in enumerate(wedges):
            w.center = center
            w.set_theta1(theta_start)
            w.set_theta2(theta_start + sweep_angle*(i+1)/n_fade)
        
        # Check if scanning is complete for current center point
        if frame_in_circle == frames_per_point - 1:  # Completed 360 degree scan
            # Count neighbors within scan radius
            neighbors = np.where(np.linalg.norm(points - center, axis=1) <= scan_radius)[0]
            if len(neighbors) >= min_points:
                if center_idx in confirmed_reachable_indices:
                    confirmed_reachable_indices.remove(center_idx)
                confirmed_core_indices.add(center_idx)
                confirmed_reachable_indices.update(neighbors)
                core_circle = Circle(
                        center,
                        scan_radius,
                        fill=False,
                        edgecolor=CIRCLE_STYLE["border_color"],
                        linewidth=CIRCLE_STYLE["border_width"],
                        zorder=3,
                    )
                ax.add_patch(core_circle)
                core_circles[center_idx] = core_circle

        # Update point colors
        new_colors = []
        for idx, (x,y) in enumerate(points):

            if idx in confirmed_core_indices:
                new_colors.append(POINT_STYLE['core_color'])
            elif idx in confirmed_reachable_indices:
                new_colors.append(POINT_STYLE['reachable_color'])
            else:
                new_colors.append(POINT_STYLE['unclassified_color'])
        scat.set_color(new_colors)
        return wedges + [scat, radar_circle] + list(core_circles.values())

    ani = FuncAnimation(fig, update, frames=total_frames, interval=5, blit=True, repeat=True)
    return ani


# -----------------------------
# Driver function
# -----------------------------
def dbscan_workflow(n_points=50, scan_radius=10, min_points=3,
                    canvas_width=150, canvas_height=100):
    points = np.column_stack((
        np.random.rand(n_points) * canvas_width,
        np.random.rand(n_points) * canvas_height,
    ))

    core_points, core_indices, points_in_circle = find_core_points(points, scan_radius, min_points)
    G = build_core_graph(core_indices, points_in_circle)
    reachable, noise = find_reachable_and_noise(points, core_indices, G, core_points, scan_radius)

    fig, axes = plt.subplots(5, 1, figsize=(10, 30))
    axes = axes.flatten()

    # Step 1
    base_axis(axes[0], canvas_width, canvas_height)
    plot_all_points(axes[0], points)
    axes[0].set_title("Step 1: All points")
    add_dynamic_legend(axes[0])

    # Step 2 with radar animation
    base_axis(axes[1], canvas_width, canvas_height)
    plot_all_points(axes[1], points)
    axes[1].set_title("Step 2: Radar scan")
    add_dynamic_legend(axes[1], show_core_indices=True, show_reachable=True)
    ani = radar_animation(ax=axes[1], points=points, min_points=min_points,
                      scan_radius=scan_radius, sweep_angle=60, sweep_speed=5,)
    animations.append(ani)
    

    # Step 3
    base_axis(axes[2], canvas_width, canvas_height)
    plot_all_points(axes[2], points, core_indices)
    plot_core_circles(axes[2], core_points, scan_radius)
    axes[2].set_title("Step 3: Core links")
    add_dynamic_legend(axes[2], show_core_indices=True, show_circles=True, show_links=True)

    # Step 4
    base_axis(axes[3], canvas_width, canvas_height)
    plot_all_points(axes[3], points, core_indices)
    plot_core_circles(axes[3], core_points, scan_radius)
    plot_core_links(axes[3], G, core_points)
    axes[3].set_title("Step 4: Connected regions")
    add_dynamic_legend(axes[3], show_core_indices=True,
                       show_circles=True, show_links=True, show_regions=True)

    # Step 5
    base_axis(axes[4], canvas_width, canvas_height)
    plot_all_points(axes[4], points, core_indices, reachable, noise)
    plot_core_links(axes[4], G, core_points)
    plot_connected_regions(axes[4], G, core_points, scan_radius)
    axes[4].set_title("Step 5: Reachable & Noise")
    add_dynamic_legend(axes[4], show_core_indices=True,
                       show_reachable=True, show_noise=True)

    plt.tight_layout()
    plt.show()

# -----------------------------
# Interactive sliders
# -----------------------------
interact_manual(
    dbscan_workflow,
    n_points=IntSlider(min=10, max=200, step=10, value=50),
    scan_radius=FloatSlider(min=2, max=30, step=1, value=10),
    min_points=IntSlider(min=1, max=10, step=1, value=3),
    canvas_width=IntSlider(min=50, max=300, step=10, value=150),
    canvas_height=IntSlider(min=50, max=300, step=10, value=100),
)
